# Directional movement analysis

In [1]:
%load_ext memory_profiler

In [2]:
import pandas as pd 
import numpy as np
import time
%memit np.zeros(100000)
%timeit np.zeros(100000)

peak memory: 98.47 MiB, increment: 0.95 MiB
13.2 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
#load data from book J.Welles Wilder
date = range(1,43)
high = [274,273.25,272,270.75,270,270.5,268.5,265.5,262.5,263.5,269.5,267.25,267.5,269.75,268.25,264,268,266,274,277.5,277,272,267.25,269.25,266,265,264.75,261,257.5,259,259.75,257.25,250,254.25,254,253.25,253.25,251.75,253,251.5,246.25,244.25]
low = [272,270.25,269.75,268,269,268,266.5,263,259,260,263,265,265.5,266,263.25,261.5,266.25,264.25,267,273.5,272.5,269.5,264,263,263.5,262,261.5,255.5,253,254,257.5,250,247,252.75,250.50,250.25,251,250.50,249.5,245.25,240,241.25]
close = [272.75,270.75,270,269.25,269.75,270,266.5,263.25,260.25,263,266.5,267,265.75,268.5,264.25,264,266.5,265.25,273,276.75,273,270.25,266.75,263,265.5,262.25,262.75,255.5,253,257.5,257.5,250,249.75,253.75,251.25,250.5,253,251.5,250,245.75,242.75,243.5]

data = {'date': date, 'high': high, 'low':low, 'close': close}

df = pd.DataFrame(data)
print(f"Dataframe shape:{df.shape}")
df

Dataframe shape:(42, 4)


,date,high,low,close
0,1,274.00,272.00,272.75
1,2,273.25,270.25,270.75
2,3,272.00,269.75,270.00
3,4,270.75,268.00,269.25
4,5,270.00,269.00,269.75
5,6,270.50,268.00,270.00
6,7,268.50,266.50,266.50
7,8,265.50,263.00,263.25
8,9,262.50,259.00,260.25
9,10,263.50,260.00,263.00


In [37]:
#load data from csv file
#df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()


Dataframe shape:(631047, 11)


,date,high,low,open,close
0,1425168000,240.00006,240.00006,240.00006,240.00006
1,1425168300,240.00006,240.00006,240.00006,240.00006
2,1425168600,240.00006,240.00006,240.00006,240.00006
3,1425168900,240.00006,240.00006,240.00006,240.00006
4,1425169200,240.00006,240.00006,240.00006,240.00006


In [31]:
def true_range(df):
    ranges = [df.high - df.low,(df.high - df.close.shift(1)).abs(),(df.low - df.close.shift(1)).abs()]
    return pd.Series(np.maximum.reduce(ranges))
  

In [32]:
def directional_movement(df):
    delta_high = df.high - df.high.shift(1)
    delta_low = df.low.shift(1) - df.low
    delta_high = np.where(delta_high < 0, 0, delta_high)
    delta_low = np.where(delta_low < 0, 0, delta_low)
    up_dm = np.where(delta_high > delta_low, delta_high, 0)
    down_dm = np.where(delta_low > delta_high, delta_low, 0)
    return pd.Series(up_dm), pd.Series(down_dm)
    

In [33]:
def average_true_range(tr,n):
    ATR = tr.ewm(com=n-1,adjust=False).mean()
    return ATR

In [55]:
def average_directional_movement_index_v0(df,n):
    tr = true_range(df)
  
    up_dm, down_dm = directional_movement(df)
    df['up_dm'] = up_dm
    df['down_dm'] = down_dm
    df['tr'] = tr
    
    up_DMn = pd.Series(up_dm)
    down_DMn = pd.Series(down_dm)
    TRn = pd.Series(tr)
    
    up_DMn[up_DMn.index[n-1]] = np.sum(up_dm[:n])
    up_DMn = up_DMn.drop(up_DMn.index[:n-1])
    
    down_DMn[down_DMn.index[n-1]] = np.sum(down_dm[:n])
    down_DMn = down_DMn.drop(down_DMn.index[:n-1])
   
    #np.sum fail when nan in numpy array if it is a pd series works ok.
    TRn[TRn.index[n-1]] = np.sum(tr[:n])
    TRn = TRn.drop(TRn.index[:n-1])
    
    for index,value in up_DMn.items():
        if index >= n :
            up_DMn[index] = (up_DMn[index-1] - (up_DMn[index-1]/14)) + up_dm[index]
    for index,value in down_DMn.items():
        if index >= n :
            down_DMn[index] = (down_DMn[index-1] - (down_DMn[index-1]/14)) + down_dm[index]
    

    for index,value in TRn.items():
        if index >= n :
            TRn[index] = (TRn[index-1] - (TRn[index-1]/14)) + tr[index]
    
    
    df['up_DMn'] = up_DMn
    df['down_DMn'] = down_DMn
    df['TRn'] = TRn
    df['up_DI'] = df.up_DMn/df.TRn
    df['down_DI'] = df.down_DMn/df.TRn

    TDM = abs(df.up_DI- df.down_DI)
    SUM_DM = df.up_DI + df.down_DI
    DX = TDM / SUM_DM

    df['DX'] = DX
    
    ADX = pd.Series(DX)
    ADX[ADX.index[(2*n) - 1]]= ADX[ADX.index[n:2*n]].mean()
    ADX = ADX.drop(ADX.index[:(2*n)-1])
    for index,value in ADX.items():
        if index >= 2*n:
            ADX[index] = ( ((n-1)*ADX[index-1]) + ADX[index] )/n
    df['ADX'] = ADX
    df['ADXR'] = (df.ADX + df.ADX.shift(n-1)) / 2
    #  actually must be n not (n-1)
    
    
    
    

In [63]:
def average_directional_movement_index_v1(df,n):
    
    df['up_dm'], df['down_dm'] = directional_movement(df)
    df['tr'] = true_range(df)   
    
    df['up_DMn'] = df.up_dm.ewm(com=n-1,adjust=False).mean()
    df['down_DMn'] = df.down_dm.ewm(com=n-1,adjust=False).mean()
    df['TRn'] = df.tr.ewm(com=n-1,adjust=False).mean()
    df['up_DI'] = df.up_DMn/df.TRn
    df['down_DI'] = df.down_DMn/df.TRn

    df['DX'] = abs(df.up_DI- df.down_DI)/(df.up_DI + df.down_DI)
    
    df['ADX'] = df.DX.ewm(com=n-1,adjust=False).mean()
    df['ADXR'] = (df.ADX + df.ADX.shift(n-1)) / 2
   


In [58]:
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df_init.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
%timeit average_directional_movement_index_v0(df,14)
#df.head(20)
df.tail(20)



Dataframe shape:(631047, 11)
135 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


,date,high,low,open,close,up_dm,down_dm,tr,up_DMn,down_DMn,TRn,up_DI,down_DI,DX,ADX,ADXR
2172,1612828800,48108.903369,45001.000000,46379.221273,46453.226939,1509.903369,0.000000,3107.903369,18053.008937,5838.357788,46337.113636,0.389601,0.125997,0.511258,0.251769,0.257480
2173,1612915200,47300.000000,43762.780000,46472.744651,44816.957036,0.000000,1238.220000,3537.220000,16763.508299,6659.552231,46564.539805,0.360006,0.143018,0.431368,0.264597,0.261252
2174,1613001600,48634.666078,44020.603105,44814.716146,47974.248884,1334.666078,0.000000,4614.062972,16900.780927,6183.869929,47852.564220,0.353184,0.129228,0.464244,0.278858,0.261141
2175,1613088000,48982.142343,46161.553135,48006.694862,47340.418911,347.476266,0.000000,2820.589209,16041.058555,5742.164934,47255.113127,0.339457,0.121514,0.472790,0.292710,0.261343
2176,1613174400,48094.123354,46275.000000,47307.970700,47150.796973,0.000000,0.000000,1819.123354,14895.268658,5332.010296,45698.871258,0.325944,0.116677,0.472790,0.305573,0.260289
2177,1613260800,49600.000000,47037.004844,47157.960771,48577.406565,1505.876646,0.000000,2562.995156,15337.197542,4951.152418,44997.661324,0.340844,0.110031,0.511922,0.320312,0.261365
2178,1613347200,48993.306600,45792.000000,48601.675122,47925.742000,0.000000,1245.004844,3201.306600,14241.683432,5842.503518,44984.849258,0.316588,0.129877,0.418199,0.327304,0.261332
2179,1613433600,50300.000000,47052.617466,47905.963549,49171.455018,1306.693400,0.000000,3247.382534,14531.113730,5425.181838,45019.028274,0.322777,0.120509,0.456294,0.336518,0.265471
2180,1613520000,52594.101000,48975.771098,49171.455008,52098.256316,2294.101000,0.000000,3618.329902,15787.278035,5037.668849,45421.713300,0.347571,0.110909,0.516189,0.349351,0.273041
2181,1613606400,52522.094105,50928.194902,52112.700374,51586.063254,0.000000,0.000000,1593.899203,14659.615318,4677.835360,43771.204410,0.334915,0.106870,0.516189,0.361268,0.280071


In [65]:
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
%timeit average_directional_movement_index_v1(df,14) # x36 faster and converge
df.tail()

Dataframe shape:(2192, 11)
3.75 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,date,high,low,open,close,up_dm,down_dm,tr,up_DMn,down_DMn,TRn,up_DI,down_DI,DX,ADX,ADXR
2187,1614124800,51347.547800,47012.372297,48893.566626,49694.850275,0.0000,0.000000,4335.175503,968.025917,874.182466,4129.412502,0.234422,0.211697,0.050941,0.348783,0.313820
2188,1614211200,52019.280000,46719.505800,49678.345446,47071.988756,671.7322,0.000000,5299.774200,946.862080,811.740861,4213.009766,0.224747,0.192675,0.076834,0.329358,0.311034
2189,1614297600,48448.570888,44122.216031,47046.151566,46310.000000,0.0000,2597.289769,4326.354856,879.229075,939.280069,4221.105844,0.208294,0.222520,0.033022,0.308191,0.306882
2190,1614384000,48388.981486,45000.000000,46320.000000,46121.948326,0.0000,0.000000,3388.981486,816.426998,872.188635,4161.668390,0.196178,0.209577,0.033022,0.288536,0.304424
2191,1614470400,46615.782170,43000.000000,46116.196781,45170.087409,0.0000,2000.000000,3615.782170,758.110784,952.746590,4122.676517,0.183888,0.231099,0.113765,0.276053,0.301678
